In [1]:
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import evaluate
from huggingface_hub import notebook_login

/home/jw/anaconda3/envs/torch2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-12 10:53:06.874945: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 10:53:06.874968: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 10:53:06.874980: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 10:53:06.878386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow 

In [2]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2).to("cuda")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
tum_dataset = load_dataset("yangezheng/tum-nlp-sexism-socialmedia-balanced")
lidia_dataset = load_dataset("yangezheng/lidiapierre-fr_sexism_labelled")
cmsb_dataset = load_dataset("yangezheng/CMSB")
swsr_comment_dataset = load_dataset("yangezheng/SWSR-SexComment")
swsr_weibo_dataset = load_dataset("yangezheng/SWSR-SexWeibo")
exist_dataset = load_dataset("yangezheng/EXIST2021")

Using custom data configuration yangezheng--tum-nlp-sexism-socialmedia-balanced-5f6a67b6df4f5ff5
Found cached dataset parquet (/home/jw/.cache/huggingface/datasets/yangezheng___parquet/yangezheng--tum-nlp-sexism-socialmedia-balanced-5f6a67b6df4f5ff5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 781.45it/s]
Using custom data configuration yangezheng--lidiapierre-fr_sexism_labelled-47db41fd986d0b6f
Found cached dataset parquet (/home/jw/.cache/huggingface/datasets/yangezheng___parquet/yangezheng--lidiapierre-fr_sexism_labelled-47db41fd986d0b6f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 1008.41it/s]
Using custom data configuration yangezheng--CMSB-e8830d4b5003421a
Found cached dataset parquet (/home/jw/.cache/huggingface/datasets/yangezheng___parquet/yangezheng--CMSB-e8830d4b5003421a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████

In [4]:
lidia_dataset = lidia_dataset.remove_columns("fr_sentences")
cmsb_dataset = cmsb_dataset.remove_columns("toxicity")
swsr_comment_dataset = swsr_comment_dataset.remove_columns("text_cn")
swsr_comment_dataset = swsr_comment_dataset.remove_columns("category")
swsr_comment_dataset = swsr_comment_dataset.remove_columns("target")
swsr_weibo_dataset = swsr_weibo_dataset.remove_columns("text_cn")
swsr_weibo_dataset = swsr_weibo_dataset.remove_columns("keyword")
exist_dataset = exist_dataset.remove_columns("text_original")
exist_dataset = exist_dataset.remove_columns("task2")

In [5]:
train_dataset = concatenate_datasets([tum_dataset["train"], lidia_dataset["train"], cmsb_dataset["train"], swsr_comment_dataset["train"], swsr_weibo_dataset["train"], exist_dataset["train"]])
train_dataset

Dataset({
    features: ['text', 'label_sexist'],
    num_rows: 41335
})

In [6]:
validation_dataset = concatenate_datasets([tum_dataset["validation"], lidia_dataset["validation"], cmsb_dataset["validation"], swsr_comment_dataset["validation"], swsr_weibo_dataset["validation"], exist_dataset["validation"]])
validation_dataset

Dataset({
    features: ['text', 'label_sexist'],
    num_rows: 5368
})

In [7]:
test_dataset = concatenate_datasets([tum_dataset["test"], lidia_dataset["test"], cmsb_dataset["test"], swsr_comment_dataset["test"], swsr_weibo_dataset["test"], exist_dataset["test"]])
test_dataset

Dataset({
    features: ['text', 'label_sexist'],
    num_rows: 8730
})

In [8]:
pandas_dataset = train_dataset.to_pandas()
pandas_dataset

,text,label_sexist
0,ITALY - RAPE JIHAD days after horrific gang-ra...,0
1,I dont take you places because I wanna be frie...,1
2,[USER] This is actually the most worrying bit ...,1
3,You mother fucking piece of trash stupid fucki...,1
4,"Yes, this will certainly bring down Trump.",0
...,...,...
41330,The pain is not in quarantine.Ref. Advance of ...,0
41331,You enter social networks and in the midst of ...,1
41332,"All my support for @gabrielsotoMEX, who is the...",1
41333,When farts are going to understand that we don...,1


In [9]:
pandas_dataset = pandas_dataset.dropna()
pandas_dataset

,text,label_sexist
0,ITALY - RAPE JIHAD days after horrific gang-ra...,0
1,I dont take you places because I wanna be frie...,1
2,[USER] This is actually the most worrying bit ...,1
3,You mother fucking piece of trash stupid fucki...,1
4,"Yes, this will certainly bring down Trump.",0
...,...,...
41330,The pain is not in quarantine.Ref. Advance of ...,0
41331,You enter social networks and in the midst of ...,1
41332,"All my support for @gabrielsotoMEX, who is the...",1
41333,When farts are going to understand that we don...,1


In [10]:
train_dataset = Dataset.from_pandas(pandas_dataset)
train_dataset

Dataset({
    features: ['text', 'label_sexist', '__index_level_0__'],
    num_rows: 41333
})

In [11]:
train_dataset = train_dataset.map(lambda e: tokenizer(e["text"], padding="max_length", truncation=True), batched=True)
train_dataset

100%|██████████| 42/42 [00:05<00:00,  8.01ba/s]


Dataset({
    features: ['text', 'label_sexist', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 41333
})

In [12]:
validation_dataset = validation_dataset.map(lambda e: tokenizer(e["text"], padding="max_length", truncation=True), batched=True)
validation_dataset

100%|██████████| 6/6 [00:00<00:00,  9.12ba/s]


Dataset({
    features: ['text', 'label_sexist', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5368
})

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", save_strategy="no")

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
train_dataset = train_dataset.rename_column("label_sexist", "label")
validation_dataset = validation_dataset.rename_column("label_sexist", "label")

In [17]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 41333
})

In [18]:
validation_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5368
})

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
    
)
trainer.train()

from huggingface_hub import login

login()
model.push_to_hub("bertweet-base-sexism")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.427000,0.442673,0.817809
2,0.377100,0.417789,0.820976
3,0.258800,0.609550,0.823584


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/jw/.cache/huggingface/token
Login successful


pytorch_model.bin: 100%|██████████| 540M/540M [35:08<00:00, 256kB/s] 


CommitInfo(commit_url='https://huggingface.co/jinx2321/bertweet-base-sexism/commit/3e4d2a1133fbeead832bbc5c02a17e609cf1882c', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='3e4d2a1133fbeead832bbc5c02a17e609cf1882c', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
from datasets import load_metric
from evaluate import evaluator

task_evaluator = evaluator("text-classification")
results = task_evaluator.compute(
    model_or_pipeline=model,
    data=test_dataset,
    tokenizer=tokenizer,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_column="label_sexist",
    label_mapping={"LABEL_0": 0, "LABEL_1": 1}
)

print(results)

{'accuracy': 0.7943871706758304, 'recall': 0.7536193735193472, 'precision': 0.7692101020956476, 'f1': 0.7613349288658422, 'total_time_in_seconds': 47.21233773799031, 'samples_per_second': 184.90929316925644, 'latency_in_seconds': 0.005408057014660975}


In [23]:
no_eval = AutoModelForSequenceClassification.from_pretrained("jinx2321/no-eval").to("cuda")

task_evaluator = evaluator("text-classification")
results = task_evaluator.compute(
    model_or_pipeline=no_eval,
    data=test_dataset,
    tokenizer=tokenizer,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_column="label_sexist",
    label_mapping={"LABEL_0": 0, "LABEL_1": 1}
)

print(results)

{'accuracy': 0.7581901489117984, 'recall': 0.7959989470913398, 'precision': 0.6935779816513762, 'f1': 0.741267312170609, 'total_time_in_seconds': 46.21313320600893, 'samples_per_second': 188.90733854991828, 'latency_in_seconds': 0.0052936005963355025}


In [24]:
work_dataset = load_dataset("lidiapierre/fr_sexism_labelled")["train"]

task_evaluator = evaluator("text-classification")
work_results = task_evaluator.compute(
    model_or_pipeline=model,
    data=work_dataset,
    tokenizer=tokenizer,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    input_column="Sentences",
    label_column="Label",
    label_mapping={"LABEL_0": 0, "LABEL_1": 1}
)

print(work_results)

Using custom data configuration lidiapierre--fr_sexism_labelled-e5562c95e5b2c5a9


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1311.95it/s]


Dataset parquet downloaded and prepared to /home/jw/.cache/huggingface/datasets/lidiapierre___parquet/lidiapierre--fr_sexism_labelled-e5562c95e5b2c5a9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 1038.19it/s]


{'accuracy': 0.9155672823218998, 'recall': 0.8846153846153846, 'precision': 0.9583333333333334, 'f1': 0.9199999999999999, 'total_time_in_seconds': 6.077365921024466, 'samples_per_second': 187.0876321708032, 'latency_in_seconds': 0.005345088760795485}
